in this notebook i will use this awesome notebook [Incredible TPUs - finetune EffNetB0-B6 at once](https://www.kaggle.com/agentauers/incredible-tpus-finetune-effnetb0-b6-at-once) for my modeling.


![](https://media.cdn.wederm.com/wp-content/uploads/sites/3/2019/06/19205822/iStock-1060795860-300x208.jpg)

Hello Everyone! i hope you are well and  enjoying things on kaggle as usual. we know that DEEP LEARNING has huge scope in healthcare but unfortunately applying them in healthcare isn't that simple and the stake is very high. IMHO this competition is more than just a CLASSIFICATION problem and if applied very carefully, it can benefit the world in enormous ways. A common warning sign of melanoma is change. Melanoma often begins in or near an existing mole. A change in the shape, color or size of a mole can be a warning sign of melanoma. Also be aware if a mole becomes painful or begins to bleed or itch.

**There are four main types of melanoma, including a rare but dangerous kind that typically affects people of color.**

![](https://images.agoramedia.com/everydayhealth/gcms/Melanoma-Types-722x406.jpg)

# Imports

In [ ]:
!pip install -q efficientnet

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, random, re, math, time
random.seed(a=42)
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import cv2
from tensorflow.keras import backend as K
import seaborn as sns
color = sns.color_palette()
%matplotlib inline

from skimage.io import imread
import efficientnet.tfkeras as efn
import PIL
from kaggle_datasets import KaggleDatasets
from tqdm import tqdm
import matplotlib.pyplot as  plt
import seaborn as sns

In [ ]:
tf.__version__ 

In [ ]:
BASEPATH = "../input/siim-isic-melanoma-classification"
df_train = pd.read_csv(os.path.join(BASEPATH, 'train.csv'))
df_test  = pd.read_csv(os.path.join(BASEPATH, 'test.csv'))
df_sub   = pd.read_csv(os.path.join(BASEPATH, 'sample_submission.csv'))


# How many samples for each class are there in the dataset?

In [ ]:
# Get the counts for each class
cases_count = df_train['target'].value_counts()
print(cases_count)

# Plot the results 
plt.figure(figsize=(10,8))
sns.barplot(x=cases_count.index, y= cases_count.values)
plt.title('Number of cases', fontsize=14)
plt.xlabel('Case type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(range(len(cases_count.index)), ['Normal(0)', 'melanoma(1)'])
plt.show()

as you can see the data is highly imbalance where only 584 positive samples and 32542 negative sample images were given so it is very easy for models to mark almost any given image as non melanoma sample and if it does that then we don't get any help from our Deep Learning model to solve this healthcare related problem,right? IMHO the main goal of a healthcare related problem while using Deep Learning / Machine learning for solving that is to **REDUCE FALSE POSITIVE(FP) RATE** .A false positive is an outcome where the model incorrectly predicts the positive class.

# Now Let's Know a Little About Melanoma

![](https://oblique.co.nz/images/hair_health/abcdes.jpg)

[**Identifying Melanoma**](https://oblique.co.nz/hair_health/melanoma.html)

The A.B.C.D. rule has been very effective at aiding the early identification of superficial spreading melanomas. Superficial spreading melanoma can have any one of the following criteria.

- Asymmetry – The shape of one half does not match the other.
- Border – The edges are often ragged, notched, blurred, or irregular in outline; the pigment may spread into the surrounding skin.
- Colour – The colour is uneven. Shades of black, brown, and tan may be present. Areas of white, grey, red, pink, or blue also may be seen.
- Diameter – Size changes and usually increases. Typically, melanomas are at least 6mm in diameter (the diameter of a pencil).

[**Deadly Melanoma Risk Factors**](https://oblique.co.nz/hair_health/melanoma.html)

*Many risk factors for melanoma have been identified. The most important risk factor is exposure to sunlight, particularly UV-B radiation. Recent studies suggest that cutaneous melanomas of the head and neck are significantly more likely to occur in people with high levels of total sun exposure. Conversely, melanomas on the trunk tend to develop on people with lower levels of ambient sunlight exposure, but who also experience higher levels of recreational exposure on the chest and back. Sunburns early in life and exposure to UV radiation from tanning beds are other factors in the development of melanoma.* **People who burn easily, such as those with fair or red hair, blue eyes, and light-coloured skin, are most prone to develop melanoma. The presence of freckling  also indicates an increased risk for melanoma development**



**[People with these risk factors have a higher probability of developing melanoma:](https://www.wederm.com/2019/06/05/the-abcde-rules-of-melanoma/)**

1. Fair, sun-sensitive skin that burns easily or tans poorly
2. Red or blond hair; Blue or green eyes
3. Have 50 or more moles
4. Unusual or irregular-looking moles that are typically large in size
5. A history of sunburns or indoor tanning bed use
6. Blood relatives who have or had melanoma
7. Immune system weakness due to disease, organ transplant or medication
8. History of melanoma or another skin cancer
9. Age 50 or older

[**Most common site for melanoma**](https://www.mdedge.com/dermatology/article/203885/melanoma/infographic-skin-cancer-stats-hispanic-patients)

![](https://iuploads.scribblecdn.net/afb5a541-2cdc-445c-a47d-49d96ed3bdf4/global/imagelib/ct_skin_cancer/revisedparts.gif)

Early detection and diagnosing melanoma can save life sometimes, if you read this [Woman diagnosed with melanoma twice in her 20s warns tanning is 'not worth it'](https://www.today.com/health/how-spot-melanoma-woman-diagnosed-skin-cancer-twice-30-t180904) then you will know that a melanoma survivor Elizabeth Hazuka saying “Getting skin checks is so important,"

![](https://media4.s-nbcnews.com/i/newscms/2020_19/1566685/skin-cancer-today-main-200506_718fb2508acc5ade6d224acb597f8dc5.jpg)

from the right side picture you can see **Doctors removed the spot and a large margin of skin to make sure all of the cancer was gone.**
just imagine that if it was not detected and diagnosed at early stage then what could happen?

# Facts

1. Melanoma is the most serious form of skin cancer
2. It makes up 2% of skin cancers, but is responsible for 75% of skin cancer deaths
3. Australia and New Zealand have highest melanoma rates in the world
4. 1 in 17 Australians will be diagnosed with melanoma before the age of 85
5. Most melanomas are caused by prolonged and repeated exposure to UV radiation in sunlight
6. More than 90% of melanoma can be successfully treated with surgery if detected early

**These facts were collated from Melanoma Patients Australia. Head over to their [website here](https://melanomapatients.org.au/about-melanoma/melanoma-facts/) for more information.**

[**Levels of Melanoma** ](http://runningalive.com/melanoma-skin-cancer-now-bed-bound-and-no-running/)

The Clark Scale has 5 levels:

1. Cells are in the out layer of the skin (epidermis)
2. Cells are in the layer directly under the epidermis (pupillary dermis)
3. The cells are touching the next layer known as the deep dermis
4. Cells have spread to the reticular dermis
5. Cells have grown in the fat layer

![](https://media.giphy.com/media/lSJElktZ5BKUvYSztq/giphy.gif)

# Diagnosis
Lucky for me my melanoma is at Level 1:)


![](http://runningalive.com/wp-content/uploads/2019/10/Clark-Scale-Level-1-Melanoma.001-480x270.jpeg)

Enough domain knowledge,now let's work on modeling section for detecting this deadly disease!

# Reading melanoma data,thank you Chris Deotte :)

In [ ]:
GCS_PATH    = KaggleDatasets().get_gcs_path('melanoma-256x256')
GCS_PATH2    = KaggleDatasets().get_gcs_path('isic2019-256x256')
files_train = tf.io.gfile.glob(GCS_PATH + '/train*.tfrec')
#files_train += tf.io.gfile.glob(GCS_PATH2 + '/train*.tfrec')
files_train1 = tf.io.gfile.glob(GCS_PATH2 + '/train*.tfrec')
files_test  = np.sort(np.array(tf.io.gfile.glob(GCS_PATH + '/test*.tfrec')))
#np.random.shuffle(files_train)

# Model configuration

In [ ]:
DEVICE = "TPU"
CFG = dict(
    batch_size        =  128,
    read_size         = 256, 
    crop_size         = 250, 
    net_size          = 248,
    LR_START          =   0.00005,
    LR_MAX            =   0.00020,
    LR_MIN            =   0.00001,
    LR_RAMPUP_EPOCHS  =   5,
    LR_SUSTAIN_EPOCHS =   0,
    LR_EXP_DECAY      =   0.8,
    
    epochs            =  5,
    
    rot               = 180.0,
    shr               =   1.5,
    hzoom             =   6.0,
    wzoom             =   6.0,
    hshift            =   6.0,
    wshift            =   6.0,
    optimizer         = 'adam',
    label_smooth_fac  =   0.05,
    tta_steps         =  25    
)

# Tpu setting

In [ ]:
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear    = math.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))


def transform(image, cfg):    
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = cfg["read_size"]
    XDIM = DIM%2 #fix for size 331
    
    rot = cfg['rot'] * tf.random.normal([1], dtype='float32')
    shr = cfg['shr'] * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / cfg['hzoom']
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / cfg['wzoom']
    h_shift = cfg['hshift'] * tf.random.normal([1], dtype='float32') 
    w_shift = cfg['wshift'] * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x   = tf.repeat(tf.range(DIM//2, -DIM//2,-1), DIM)
    y   = tf.tile(tf.range(-DIM//2, DIM//2), [DIM])
    z   = tf.ones([DIM*DIM], dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM//2+XDIM+1, DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([DIM//2-idx2[0,], DIM//2-1+idx2[1,]])
    d    = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM, DIM,3])

In [ ]:
def read_labeled_tfrecord(example):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string),
        'patient_id'                   : tf.io.FixedLenFeature([], tf.int64),
        'sex'                          : tf.io.FixedLenFeature([], tf.int64),
        'age_approx'                   : tf.io.FixedLenFeature([], tf.int64),
        'anatom_site_general_challenge': tf.io.FixedLenFeature([], tf.int64),
        'diagnosis'                    : tf.io.FixedLenFeature([], tf.int64),
        'target'                       : tf.io.FixedLenFeature([], tf.int64)
    }           
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['target']


def read_unlabeled_tfrecord(example, return_image_name):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['image_name'] if return_image_name else 0

 
def prepare_image(img, cfg=None, augment=True):    
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [cfg['read_size'], cfg['read_size']])
    img = tf.cast(img, tf.float32) / 255.0
    
    if augment:
        img = transform(img, cfg)
        img = tf.image.random_crop(img, [cfg['crop_size'], cfg['crop_size'], 3])
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_hue(img, 0.01)
        img = tf.image.random_saturation(img, 0.7, 1.3)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.1)

    else:
        img = tf.image.central_crop(img, cfg['crop_size'] / cfg['read_size'])
                                   
    img = tf.image.resize(img, [cfg['net_size'], cfg['net_size']])
    img = tf.reshape(img, [cfg['net_size'], cfg['net_size'], 3])
    return img

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) 
         for filename in filenames]
    return np.sum(n)


In [ ]:
def get_dataset(files, cfg, augment = False, shuffle = False, repeat = False, 
                labeled=True, return_image_names=True):
    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = ds.cache()
    
    if repeat:
        ds = ds.repeat()
    
    if shuffle: 
        ds = ds.shuffle(1024*8)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
        
    if labeled: 
        ds = ds.map(read_labeled_tfrecord, num_parallel_calls=AUTO)
    else:
        ds = ds.map(lambda example: read_unlabeled_tfrecord(example, return_image_names), 
                    num_parallel_calls=AUTO)      
    
    ds = ds.map(lambda img, imgname_or_label: (prepare_image(img, augment=augment, cfg=cfg), 
                                               imgname_or_label), 
                num_parallel_calls=AUTO)
    
    ds = ds.batch(cfg['batch_size'] * REPLICAS)
    ds = ds.prefetch(AUTO)
    return ds

**Let's look at how a normal case is different from that of a melanoma case. We will look at somes samples from external data of alex shonenkov.**

thank you alex shonenkov


In [ ]:
# Get few samples for both the classes
shonenkovData = pd.read_csv('../input/melanoma-merged-external-data-512x512-jpeg/folds.csv')
melanoma_samples = (shonenkovData[shonenkovData['target']==1]['image_id'].iloc[:5]).tolist()
normal_samples = (shonenkovData[shonenkovData['target']==0]['image_id'].iloc[:5]).tolist()
# Concat the data in a single list and del the above two list
samples = melanoma_samples + normal_samples
del melanoma_samples, normal_samples
source = "../input/melanoma-merged-external-data-512x512-jpeg/512x512-dataset-melanoma/512x512-dataset-melanoma/"
# Plot the data 
f, ax = plt.subplots(2,5, figsize=(30,10))
for i in range(10):
    img = imread(source + samples[i]+".jpg")
    ax[i//5, i%5].imshow(img, cmap='gray')
    if i<5:
        ax[i//5, i%5].set_title("melanoma")
    else:
        ax[i//5, i%5].set_title("Normal")
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_aspect('auto')
plt.show()

i am not a radiologist,but correct me if i am wrong, using my naked eyes it looks like some normal cases more  like melanoma cases  hence making it more difficult for models to generalize?

**now let's have a look at competitions resized data of chris data and try to compare them with external data**

# Train samples

In [ ]:
def show_dataset(thumb_size, cols, rows, ds):
    mosaic = PIL.Image.new(mode='RGB', size=(thumb_size*cols + (cols-1), 
                                             thumb_size*rows + (rows-1)))
   
    for idx, data in enumerate(iter(ds)):
        img, target_or_imgid = data
        ix  = idx % cols
        iy  = idx // cols
        img = np.clip(img.numpy() * 255, 0, 255).astype(np.uint8)
        img = PIL.Image.fromarray(img)
        img = img.resize((thumb_size, thumb_size), resample=PIL.Image.BILINEAR)
        mosaic.paste(img, (ix*thumb_size + ix, 
                           iy*thumb_size + iy))

    display(mosaic)
    
ds = get_dataset(files_train, CFG).unbatch().take(12*2)   
show_dataset(512, 12, 2, ds)

In [ ]:
ds = tf.data.TFRecordDataset(files_train, num_parallel_reads=AUTO)
ds = ds.take(1).cache().repeat()
ds = ds.map(read_labeled_tfrecord, num_parallel_calls=AUTO)
ds = ds.map(lambda img, target: (prepare_image(img, cfg=CFG, augment=True), target), 
            num_parallel_calls=AUTO)
ds = ds.take(12*2)
ds = ds.prefetch(AUTO)

show_dataset(512, 12, 2, ds)

# test samples

In [ ]:
ds = get_dataset(files_test, CFG, labeled=False).unbatch().take(12*2)   
show_dataset(512, 12, 2, ds)

**to me it seems like external data is little bit different than our competitions data,what  do you think?**

now let's visualize some of external data that chris deotte prepared,thanks to [this post](https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/164910) i will visualize some samples from [this data](https://www.kaggle.com/cdeotte/isic2019-256x256)

In [ ]:


# Get few samples for both the classes
chrisData = pd.read_csv('../input/jpeg-isic2019-256x256/train.csv')
melanoma_samples = (chrisData[chrisData['target']==1]['image_name'].iloc[:5]).tolist()
normal_samples = (chrisData[chrisData['target']==0]['image_name'].iloc[:5]).tolist()
# Concat the data in a single list and del the above two list
samples = melanoma_samples + normal_samples
del melanoma_samples, normal_samples
source = "../input/jpeg-isic2019-256x256/train/"
# Plot the data 
f, ax = plt.subplots(2,5, figsize=(30,10))
for i in range(10):
    img = imread(source + samples[i]+".jpg")
    ax[i//5, i%5].imshow(img, cmap='gray')
    if i<5:
        ax[i//5, i%5].set_title("melanoma")
    else:
        ax[i//5, i%5].set_title("Normal")
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_aspect('auto')
plt.show()

Again with naked eyes i can see some normal cases looks like melanoma! your thought?

# Depthwise Separable Convolution in Xception

this is the most important section, let's start with a simple yet powerful network called Xception  


# Understanding Depthwise Separable Convolution

![](https://miro.medium.com/max/1050/1*VvBTMkVRus6bWOqrK1SlLQ.png)

**Above Figure : Original Depthwise Separable Convolution**

The original depthwise separable convolution is the depthwise convolution followed by a pointwise convolution.

1. Depthwise convolution is the channel-wise n×n spatial convolution. Suppose in the figure above, we have 5 channels, then we will have 5 n×n spatial convolution.

2. Pointwise convolution actually is the 1×1 convolution to change the dimension.

Compared with conventional convolution, we do not need to perform convolution across all channels. That means the number of connections are fewer and the model is lighter.


![](https://miro.medium.com/max/1050/1*J8dborzVBRBupJfvR7YhuA.png)

**The Modified Depthwise Separable Convolution used as an Inception Module in Xception, so called “extreme” version of Inception module (n=3 here)**

The modified depthwise separable convolution is the pointwise convolution followed by a depthwise convolution. This modification is motivated by the inception module in Inception-v3 that 1×1 convolution is done first before any n×n spatial convolutions. Thus, it is a bit different from the original one. (n=3 here since 3×3 spatial convolutions are used in Inception-v3.)

**Two minor differences:**
1. The order of operations: As mentioned, the original depthwise separable convolutions as usually implemented (e.g. in TensorFlow) perform first channel-wise spatial convolution and then perform 1×1 convolution whereas the modified depthwise separable convolution perform 1×1 convolution first then channel-wise spatial convolution. This is claimed to be unimportant because when it is used in stacked setting, there are only small differences appeared at the beginning and at the end of all the chained inception modules.

2. The Presence/Absence of Non-Linearity: In the original Inception Module, there is non-linearity after first operation. In Xception, the modified depthwise separable convolution, there is NO intermediate ReLU non-linearity.

![](https://miro.medium.com/max/773/1*x99M0RaK9i2X9kYrEYtYdg.png)

The modified depthwise separable convolution with different activation units are tested. As from the above figure, the Xception without any intermediate activation has the highest accuracy compared with the ones using either ELU or ReLU.

information source : [Review: Xception — With Depthwise Separable Convolution, Better Than Inception-v3 (Image Classification)](https://towardsdatascience.com/review-xception-with-depthwise-separable-convolution-better-than-inception-v3-image-dc967dd42568)

# callback - Learning rate finder

In [ ]:
def get_lr_callback(cfg):
    lr_start   = cfg['LR_START']
    lr_max     = cfg['LR_MAX'] * strategy.num_replicas_in_sync
    lr_min     = cfg['LR_MIN']
    lr_ramp_ep = cfg['LR_RAMPUP_EPOCHS']
    lr_sus_ep  = cfg['LR_SUSTAIN_EPOCHS']
    lr_decay   = cfg['LR_EXP_DECAY']
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

[**Multi-Sample Dropout**](https://www.kaggle.com/doanquanvietnamca/multi-dropout-kfold-with-roberta-on-gpu)

Dropout is an efficient regularization instrument for avoiding overfitting of deep neural networks. It works very simply randomly discarding a portion of neurons during training; as a result, a generalization occurs because in this way neurons depend no more on each other. In this post, I try to reproduce the results presented in this paper; which introduced a technique called Multi-Sample Dropout. As declared by the author, its scopes are:

* accelerate training and improve generalization over the original dropout

* reduce computational cost because most of the computation time is consumed in the layers below (often convolutional or recurrent) and the weights in the layers at the top are shared

* achieve lower error rates and losses.

![](https://miro.medium.com/max/700/1*mdBXp3-D7G7mTcKDZHui8g.png)

When a particular problem includes an imbalanced dataset, then accuracy isn't a good metric to look for. For example, if your dataset contains 95 negatives and 5 positives, having a model with 95% accuracy doesn't make sense at all. The classifier might label every example as negative and still achieve 95% accuracy. Hence, we need to look for alternative metrics. Precision and Recall are really good metrics for such kind of problems.

We will get the confusion matrix from our predictions and see what is the recall and precision of our model.

# Model : will use VGG-16 from [Beating everything with Depthwise Convolution](https://www.kaggle.com/aakashnain/beating-everything-with-depthwise-convolution)

thank you @aakashnain :)

i really loved this [**Skin Lesion Detection**](https://github.com/AakashKumarNain/skin_cancer_detection) android app of yours :) 


This is the best part. If you look at other kernels on this dataset, everyone is busy doing transfer learning and fine-tuning. You should transfer learn but wisely. We will be doing partial transfer learning and rest of the model will be trained from scratch. I will explain this in detail but before that, I would love to share one of the best practices when it comes to building deep learning models from scratch on limited data.

1. Choose a simple architecture.

2. Initialize the first few layers from a network that is pretrained on imagenet. This is because first few layers capture general details like color blobs, patches, edges, etc. Instead of randomly initialized weights for these layers, it would be much better if you fine tune them.

3. Choose layers that introduce a lesser number of parameters. For example, Depthwise SeparableConv is a good replacement for Conv layer. It introduces lesser number of parameters as compared to normal convolution and as different filters are applied to each channel, it captures more information. Xception a powerful network, is built on top of such layers only. You can read about Xception and Depthwise Separable Convolutions in [this paper.](https://arxiv.org/abs/1610.02357)

4. Use batch norm with convolutions. As the network becomes deeper, batch norm start to play an important role.

5. Add multi sample dropout. Train with a higher learning rate and experiment with the number of neurons in the dense layers. Do it for the depth of your network too.

6. Once you know a good depth, start training your network with a lower learning rate along with decay.

This is all that I have done in the next code block.

**update : i changed maxpooling layer and dropouts with multisample dropouts**

In [ ]:
#https://www.kaggle.com/aakashnain/beating-everything-with-depthwise-convolution
def build_model():
    input_img = Input(shape=(256,256,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)
    
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)
    
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)
    
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
    x = BatchNormalization(name='bn3')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
    x = BatchNormalization(name='bn4')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
    x = MaxPooling2D((3,3), name='pool4')(x)
    
    x = Flatten(name='flatten')(x)


    dense = []
    for p in np.linspace(0.5,0.7, 5):
        x = Dense(1024, activation='relu', name= f'fc1{p}')(x)
        x_ = tf.keras.layers.Dropout(p)(x)
        dense.append(x_)
    x = tf.keras.layers.Average()(dense)
    

    dense = []
    for p in np.linspace(0.3,0.5, 5):
        x = Dense(512, activation='relu', name= f'fc2{p}')(x)
        x_ = tf.keras.layers.Dropout(p)(x)
        dense.append(x_)
    x = tf.keras.layers.Average()(dense)
    
    
    x = Dense(1, activation='sigmoid', name='fc3')(x)
    
    model = Model(inputs=input_img, outputs=x)
    return model

In [ ]:
model =  build_model()
model.summary()

We will initialize the weights of first two convolutions with imagenet weights,

In [ ]:
# Open the VGG16 weight file
#https://www.kaggle.com/aakashnain/beating-everything-with-depthwise-convolution
import h5py
f = h5py.File('../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 'r')

# Select the layers for which you want to set weight.

w,b = f['block1_conv1']['block1_conv1_W_1:0'], f['block1_conv1']['block1_conv1_b_1:0']
model.layers[1].set_weights = [w,b]

w,b = f['block1_conv2']['block1_conv2_W_1:0'], f['block1_conv2']['block1_conv2_b_1:0']
model.layers[2].set_weights = [w,b]

w,b = f['block2_conv1']['block2_conv1_W_1:0'], f['block2_conv1']['block2_conv1_b_1:0']
model.layers[4].set_weights = [w,b]

w,b = f['block2_conv2']['block2_conv2_W_1:0'], f['block2_conv2']['block2_conv2_b_1:0']
model.layers[5].set_weights = [w,b]

f.close()
model.summary()  

In [ ]:
model.compile(
            optimizer = CFG['optimizer'],
            loss      =  tf.keras.losses.BinaryCrossentropy(label_smoothing = CFG['label_smooth_fac']),
            metrics   = [tf.keras.metrics.AUC(name='auc')])

# Running for 40 epochs/fold

In [ ]:
%%time
# from : https://www.kaggle.com/cdeotte/triple-stratified-kfold-with-tfrecords/comments?
skf = KFold(n_splits=2,shuffle=True,random_state=42)
oof_pred = []; oof_tar = []; oof_val = []; oof_names = [] 

GCS_PATH = [None]*2; IMG_SIZES = [256,256]

for i,k in enumerate(IMG_SIZES):
    GCS_PATH[i] = KaggleDatasets().get_gcs_path('melanoma-%ix%i'%(k,k))
    #GCS_PATH2[i] = KaggleDatasets().get_gcs_path('isic2019-%ix%i'%(k,k))
files_train = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[0] + '/train*.tfrec')))
files_test  = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[0] + '/test*.tfrec')))

for fold,(idxT,idxV) in enumerate(skf.split(np.arange(15))):
    
    # DISPLAY FOLD INFO
    if DEVICE=='TPU':
        if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)
    print('_'*25); print('**** FOLD',fold+1)
    # CREATE TRAIN AND VALIDATION SUBSETS
    files_train = tf.io.gfile.glob([GCS_PATH[fold] + '/train%.2i*.tfrec'%x for x in idxT])
    files_valid = tf.io.gfile.glob([GCS_PATH[fold] + '/train%.2i*.tfrec'%x for x in idxV])
    
    # SAVE BEST MODEL EACH FOLD
    sv = tf.keras.callbacks.ModelCheckpoint(
        'vgg16fold-%i.h5'%fold, monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True, mode='min', save_freq='epoch')
    
    steps_per_epoch  = count_data_items(files_train) / (CFG['batch_size'] * REPLICAS)

    history_Xception1   = model.fit(
        get_dataset(files_train, CFG,augment=True, shuffle=True, repeat=True), 
        epochs=40, callbacks = [sv,get_lr_callback(CFG)], 
        steps_per_epoch=count_data_items(files_train)/128//REPLICAS,
        validation_data=get_dataset(files_valid,CFG,augment=False,shuffle=False,
                repeat=False), 
        verbose=1
    )
    # PREDICT OOF USING TTA
    TTA = 2
    print('Predicting OOF with TTA...')
    ds_valid = get_dataset(files_valid,CFG,labeled=False,return_image_names=False,augment=True,
            repeat=True,shuffle=False,)
    ct_valid = count_data_items(files_valid); 
    STEPS = TTA * ct_valid/128/REPLICAS
    pred = model.predict(ds_valid,steps=STEPS,verbose=1)[:TTA*ct_valid,] 

    oof_pred.append( np.mean(pred.reshape((ct_valid,TTA),order='F'),axis=1) )                 

    # GET OOF TARGETS AND NAMES
    ds_valid = get_dataset(files_valid,CFG, augment=False, repeat=False,
            labeled=True, return_image_names=True)
    oof_tar.append( np.array([target.numpy() for img, target in iter(ds_valid.unbatch())]) )

    # REPORT RESULTS
    auc = roc_auc_score(oof_tar[-1],oof_pred[-1])
    oof_val.append(np.max( history_Xception1.history['val_auc'] ))
    print('#### FOLD %i OOF AUC with TTA = %.3f, without TTA = %.3f'%(fold+1,auc,oof_val[-1]))
    print()   
    
    
    for i in range(len(oof_pred[0])):
        if(oof_pred[0][i] > 0.5):
            oof_pred[0][i] = 1
        else:
            oof_pred[0][i] = 0
        
    oof_preds = []
    oof_tars = []
    for i in range(len(oof_pred[0])):
        oof_preds.append(oof_pred[0][i])
        oof_tars.append(oof_tar[0][i])


    # Get the confusion matrix
    cm  = confusion_matrix(oof_tars, oof_preds)
    plt.figure()
    plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True)
    plt.yticks(range(2), ['Normal', 'melanoma'], fontsize=16)
    plt.show()
    
    # Calculate Precision and Recall
    tn, fp, fn, tp = cm.ravel()

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)

    print("Recall of the model is {:.2f}".format(recall))
    print("Precision of the model is {:.2f}".format(precision))

    

In [ ]:
# COMPUTE OVERALL OOF AUC
oof = np.concatenate(oof_pred)
true = np.concatenate(oof_tar)
auc = roc_auc_score(true,oof)
print('Overall OOF AUC with TTA = %.3f'%auc)

# SAVE OOF TO DISK
df_oof = pd.DataFrame(dict( pred = oof, target=true))
df_oof.to_csv('oof.csv',index=False)
df_oof.head()

In [ ]:
oof_tars = []

oof_preds = []

for i in range(len(df_oof)):
    if(df_oof.pred[i] > 0.5):
        oof_preds.append(1)
        oof_tars.append(df_oof.target[i])
    else:
        oof_preds.append(0)
        oof_tars.append(df_oof.target[i])


# Get the confusion matrix
cm  = confusion_matrix(oof_tars, oof_preds)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True)
plt.yticks(range(2), ['Normal', 'melanoma'], fontsize=16)
plt.show()

# Calculate Precision and Recall
tn, fp, fn, tp = cm.ravel()

precision = tp/(tp+fp)
recall = tp/(tp+fn)

print("Overall Recall of the model is {:.2f}".format(recall))
print("Overall Precision of the model is {:.2f}".format(precision))

In [ ]:
history_Xception1.history['lr']

# Predicting

In [ ]:
%%time
CFG['batch_size'] = 256

cnt_test   = count_data_items(files_test)
steps      = cnt_test / (CFG['batch_size'] * REPLICAS) * CFG['tta_steps']
ds_testAug = get_dataset(files_test, CFG, augment=True, repeat=True, 
                         labeled=False, return_image_names=False)
preds = model.predict(ds_testAug, verbose=1, steps=steps)


In [ ]:
print("Test shape :",df_sub.shape)
print("Preds shape :",preds.shape)

In [ ]:
# https://www.kaggle.com/vbhargav875/efficientnet-b5-b6-b7-tf-keras

preds = preds[:,:cnt_test* CFG['tta_steps']]
preds = preds[:df_test.shape[0]*CFG['tta_steps']]
preds = np.stack(np.split(preds, CFG['tta_steps']),axis=1)
preds = np.mean(preds, axis=1)
preds = preds.reshape(-1)

In [ ]:
print("New Preds shape :",preds.shape)

# Submission

In [ ]:
ds = get_dataset(files_test, CFG, augment=False, repeat=False, 
                 labeled=False, return_image_names=True)

image_names = np.array([img_name.numpy().decode("utf-8") 
                        for img, img_name in iter(ds.unbatch())])

In [ ]:
submission = pd.DataFrame(dict(
        image_name = image_names,
        target = preds ))

submission = submission.sort_values('image_name') 
submission.to_csv(f'submission_vgg16.csv', index=False)

on tpu i get [this error](https://github.com/tensorflow/tensorflow/issues/8471)

so running on gpu just to share with you all,i think training for more epochs will be good!